In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import pickle

In [2]:
data_dir = '/opt/ml/project/data/'
train_path = os.path.join(data_dir, 'train_data.csv') 
train_df = pd.read_csv(train_path, parse_dates=['Timestamp']) 

test_path = os.path.join(data_dir, 'test_data.csv') 
test_df = pd.read_csv(test_path, parse_dates=['Timestamp']) 

df = pd.concat([train_df, test_df])
df = df.sort_values(by=['userID','Timestamp']).reset_index(drop=True)

In [4]:
elapsed_time = df.Timestamp.shift(-1) - df.Timestamp
elapsed_time = elapsed_time.values / np.timedelta64(1, 's')
df['elapsedTime'] = elapsed_time
df.loc[df.groupby('userID', as_index=False).nth(-1).index, 'elapsedTime'] = np.NaN

In [5]:
df['test_cumcount'] = df.groupby(['userID', 'testId']).cumcount() + 1
temp_df = df.groupby(['userID', 'testId'],as_index=False).assessmentItemID.nunique()
temp_df.columns = ['userID', 'testId', 'test_itemnum']
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId'])
df['test_num'] = (df['test_cumcount'] - 1) // df['test_itemnum']

In [6]:
temp_df = df[df['test_cumcount'] % df['test_itemnum'] == 1]
temp_df = temp_df[['userID', 'testId', 'test_num', 'Timestamp']]
temp_df.columns = ['userID', 'testId', 'test_num', 'test_start_time']
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId', 'test_num'])

temp_df = df[df['test_cumcount'] % df['test_itemnum'] == 0]
temp_df = temp_df[['userID', 'testId', 'test_num', 'Timestamp']]
temp_df.columns = ['userID', 'testId', 'test_num', 'test_end_time']
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId', 'test_num'])

In [7]:
cond1 = df.userID.shift(-1) == df.userID
cond2_1 = df.test_start_time < df.test_start_time.shift(-1)
cond2_2 = df.test_end_time > df.test_start_time.shift(-1)
cond3 = df.test_cumcount % df.test_itemnum == 0
cond4 = df.elapsedTime > 60

df.loc[~(cond1&cond2_1&cond2_2)&cond3&cond4, 'elapsedTime'] = np.NaN

In [8]:
def out_zscore(data):
    z_scores = []
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_scores.append((i - mean)/std)
    return np.where(np.abs(z_scores) > threshold)[0]

In [9]:
outliers_idx = out_zscore(df.elapsedTime)
df.loc[outliers_idx, 'elapsedTime'] = np.NaN

In [10]:
df['testType'] = df['testId'].apply(lambda x:x[:3])
df['testNumber'] = df['testId'].apply(lambda x:x[-3:])
df['questionNumber'] = df['assessmentItemID'].apply(lambda x:x[-3:])

drop_cols = ['test_cumcount','test_itemnum','test_num','test_start_time','test_end_time']
df = df.drop(columns=drop_cols)

In [12]:
csv_save_path = os.path.join(data_dir, 'total_data.csv')
df.to_csv(csv_save_path, index=False)